In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

# **Telegram**

In [2]:
!pip install telethon --quiet

In [3]:
from telethon.sync import TelegramClient
from telethon import events
 
import csv
 
from telethon.tl.functions.messages import GetDialogsRequest
from telethon.tl.types import InputPeerEmpty

In [4]:
import re

# removes emojis and other 'strange' symbols. Leaves punctuation
def clean_text(string):
    # all characters that may be used in ads, i think..
    reg = re.compile('[^a-zA-Zа-яА-ЯёЁ0-9\n\t~!@#$%^&*()-_=+\\|/<>,.?;:\'"№[]{} ]')
    return reg.sub('', string)

def remove_urls(string):
    return re.sub(r'\(?https?://\S*', '', string)

# checks if text contains numers
def count_numbers(string):
    string = re.sub(r'([0-9])[x\'.,]([0-9])', r'\1\2', string)
    string = re.sub(r'([a-zA-zа-яА-ЯёЁ])[0-9]+', r'\1', string)
    reg = re.compile('[0-9]+')
    return len(reg.split(string)) - 1

In [5]:
api_id = 12345678  # Your api ID
api_hash = "your_api_hash"
name = "parsing"

In [6]:
client = TelegramClient(name, api_id, api_hash, system_version="4.16.30-vxCUSTOM")

In [7]:
await client.start()

In [8]:
source = "'platform': 'telegram', 'wall_id': {}, 'post_id': {}"
source.format(2, 3)

"'platform': 'telegram', 'wall_id': 2, 'post_id': 3"

In [9]:
async def scrap_channel(channel_name, tg_client, *, ads_limit=1e5, after_date=None, log_frequency=None):
    """
    channel_name: str - name of the channel to scrap
    tg_client: TelegramClient - client to be used for scrapping
    ads_limit: int - max number of ads to get from the channel
    after_date: datetime.datetime - if provided, ads posted before this date will be skipped
    log_frequency: int - if provided, every <log_frequency> ads, total number of read messages will be printed
    """
    successfullty_read = 0
    messages, identifiers = [], []

    async for message in tg_client.iter_messages(channel_name, offset_date=datetime.date.today()):
        if not message.message:
            continue
        msg = clean_text(remove_urls(message.message))
        if len(list(msg.split())) >= 2 and count_numbers(msg) > 0:
            messages.append(msg.lower().strip())
            identifiers.append("{" + source.format(message.peer_id.channel_id, message.id) + "}")
            
            successfullty_read += 1
            if log_frequency and successfullty_read % log_frequency == 0:
                print(successfullty_read)
            if ads_limit and successfullty_read > ads_limit:
                break
            if after_date and message.date < after_date:
                break
    return messages, identifiers

In [10]:
channels = ["barakholkam", "spb_barakholka", "baraholkasppb", "baraholka_batumi_home", "baraholka_tbi", "CypRusSale"]

In [11]:
timezone_offset = 3.0
tzinfo = datetime.timezone(datetime.timedelta(hours=timezone_offset))
min_date = datetime.datetime.fromisoformat('2020-01-01T00:00:00+03:00')

messages, identifiers = [], []
for channel in channels:
    try:
        print("Start parsing", channel)
        channel_messages, channel_identifiers = await scrap_channel(channel, client, after_date=min_date, log_frequency=10000)
        messages += channel_messages
        identifiers += channel_identifiers
        print(f"After observing {channel}, we have", len(messages), "ads")
    except Exception as e:
        print(e)
        print("Something wrong with", channel)
        pass

Start parsing barakholkam
After observing barakholkam, we have 3333 ads
Start parsing spb_barakholka
After observing spb_barakholka, we have 5696 ads
Start parsing baraholkasppb
10000
After observing baraholkasppb, we have 20862 ads
Start parsing baraholka_batumi_home
10000
20000
30000
40000
50000
After observing baraholka_batumi_home, we have 74506 ads
Start parsing baraholka_tbi
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
After observing baraholka_tbi, we have 174507 ads
Start parsing CypRusSale
10000
20000
30000
40000
50000
After observing CypRusSale, we have 226990 ads


In [13]:
df = pd.DataFrame({
    "Text": messages,
    "Id": identifiers,
})

df.drop_duplicates(subset="Text", inplace=True)

df.to_csv(f"../data/telegram_ads_v2.csv", index=False)

Server closed the connection: [WinError 10054] An existing connection was forcibly closed by the remote host
Attempt 1 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 2 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 3 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 4 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 5 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 6 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 1 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 2 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
Attempt 3 at connecting failed: OSError: [Errno 10065] Connect call failed ('149.154.167.51', 443)
